In [27]:
import urllib.parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import time
from bs4 import BeautifulSoup
import re

In [28]:
def configure_driver():
    chrome_options = Options()
    # options.add_argument('--headless')  # Menjalankan Chrome tanpa antarmuka grafis
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36")
    chrome_options.add_argument('--disable-dev-shm-usage')  # Mencegah masalah memori
    chrome_options.add_argument('--disable-gpu')  # Opsi tambahan untuk meningkatkan stabilitas
    chrome_options.add_argument('--remote-debugging-port=9222')

    # Tentukan path ChromeDriver
    service = Service(r'chromedriver-win64\chromedriver.exe')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

In [31]:
# Setup WebDriver
driver = configure_driver()

# Navigasi ke halaman yang diinginkan
driver.get("https://www.indonesia-investments.com/id/bisnis/profil-perusahaan/item74")

# List untuk menyimpan data
all_data = []

# Set counter klik
click_count = 0

# Loop untuk scrape data selama belum mencapai halaman 15
while click_count < 15:
    # Tunggu hingga tabel dimuat
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "companyfiles")))

    # Ambil halaman HTML
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Cari semua tabel yang memiliki class 'companyfiles'
    table = soup.find('table', class_='companyfiles')
    rows = table.find_all('tr')  # Ambil semua baris dalam tabel

    # List untuk menyimpan data halaman ini
    page_data = []

    # Loop untuk mengambil data dari setiap baris tabel
    for row in rows:
        company_name = row.find('td', class_='companyname')
        if company_name:
            company_name = company_name.get_text(strip=True)
            
            # Ambil sektor
            sector = row.find('td', class_='sector')
            sector = sector.get_text(strip=True) if sector else 'No sector'
            
            # Ambil link website
            website = row.find('td', class_='website').find('a')['href']
            
            # Simpan data ke dalam list
            page_data.append({
                "Nama Perusahaan": company_name,
                "Sektor": sector,
                "Situs": website
            })

    # Tambahkan data halaman ini ke list utama
    all_data.extend(page_data)

    # Cari tombol "Next"
    next_button = driver.find_element(By.CLASS_NAME, 'next')

    # Mengecek apakah tombol Next aktif atau tidak
    if "disabled" in next_button.get_attribute("class"):
        print("Tombol Next tidak aktif, sudah mencapai halaman terakhir.")
        break  # Jika tombol Next disabled, berarti kita sudah di halaman terakhir

    # Menggunakan ActionChains untuk memastikan kita mengklik tombol dengan benar
    actions = ActionChains(driver)
    
    try:
        # Scroll ke tombol Next agar bisa terlihat dan diklik
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        
        # Tunggu hingga tombol Next dapat diklik
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(next_button))
        
        # Klik tombol Next
        actions.move_to_element(next_button).click().perform()
        
        # Tunggu sebentar untuk memastikan halaman berikutnya dimuat
        time.sleep(2)

        # Increment click_count
        click_count += 1
        print(f"Halaman {click_count} telah dimuat.")

    except Exception as e:
        print(f"Tidak dapat mengklik tombol Next atau sudah mencapai halaman terakhir. Error: {str(e)}")
        break  # Jika terjadi kesalahan atau tombol Next tidak bisa diklik, berhenti

# Buat DataFrame dari list all_data
df = pd.DataFrame(all_data)

# Menutup browser setelah selesai
driver.quit()

Halaman 1 telah dimuat.
Halaman 2 telah dimuat.
Halaman 3 telah dimuat.
Halaman 4 telah dimuat.
Halaman 5 telah dimuat.
Halaman 6 telah dimuat.
Halaman 7 telah dimuat.
Halaman 8 telah dimuat.
Halaman 9 telah dimuat.
Halaman 10 telah dimuat.
Halaman 11 telah dimuat.
Halaman 12 telah dimuat.
Halaman 13 telah dimuat.
Halaman 14 telah dimuat.
Halaman 15 telah dimuat.


In [32]:
df.duplicated().sum()

np.int64(0)

In [35]:
df.to_excel('indonesia-investments-scrapper.xlsx', index=False)